librerie da installare:

In [ ]:
!pip install backtrader
!pip install matplotlib==3.2.2
!pip install pandas
!pip install tqdm
!pip install ipywidgets

Carico la strategia:

In [ ]:
import backtrader as bt


class BreakIn(bt.Strategy):
    
    params = (
        ('atrPeriodo', 14),
        ('atrMult', 2),
        ('stopLoss', 0.0030),
        ('takeProfit', 0.0050),
    )

    def __init__(self):
        self.atr = bt.ind.ATR(period=self.params.atrPeriodo)
        self.openPrice = self.datas[0].open
        self.highPrice = self.datas[0].high
        self.lowPrice = self.datas[0].low
        self.closePrice = self.datas[0].close

    def next(self):
        if abs(self.openPrice - self.closePrice) > (self.atr * self.params.atrMult) and self.closePrice > self.openPrice and self.position.size == 0 :
            stopLoss = self.closePrice - self.params.stopLoss
            takeProfit = self.closePrice + self.params.takeProfit
            self.buy_bracket(size=50000, stopprice=stopLoss, limitprice=takeProfit, exectype=bt.Order.Market)

        elif abs(self.openPrice - self.closePrice) > (self.atr * self.params.atrMult) and self.closePrice < self.openPrice and self.position.size == 0:
            stopLoss = self.closePrice + self.params.stopLoss
            takeProfit = self.closePrice - self.params.takeProfit
            self.sell_bracket(size=50000, stopprice=stopLoss, limitprice=takeProfit, exectype=bt.Order.Market)

Carico e imposto il motore cerebro con la callback per ogni ottimizzazione:

In [ ]:
import backtrader as bt
from datetime import datetime
import numpy as np
import pandas as pd
from backtrader import CommInfoBase
from tqdm import tqdm


def optimizer_callbacks(cb):
    pbar.update()


def ottimizzazioneStrategia(**kwargs):
    total = np.prod([len(value) for key, value in kwargs.items()])

    initCapital = 10000
    cerebro = bt.Cerebro()
    cerebro.broker.setcommission(leverage=30, commission=0.00001, commtype=CommInfoBase.COMM_FIXED)
    cerebro.broker.set_slippage_fixed(0.0001)
    data = bt.feeds.GenericCSVData(
        dataname='FXCM_EURUSD_1H_20170101_20211013.csv',
        timeframe=bt.TimeFrame.Minutes,
        compression=60,
        fromdate=datetime(2017, 1, 1),
        todate=datetime(2019, 1, 1),
        dtformat=('%Y%m%d'),
        tmformat=('%H%M'),
        date=0,
        time=1,
        open=2,
        high=3,
        low=4,
        close=5,
        volume=6,
        openinterest=-1
    )
    # Add the data to Cerebro
    cerebro.adddata(data)
    # Set our desired cash start
    cerebro.broker.setcash(initCapital)

    cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="trade")

    strats = cerebro.optstrategy(BreakIn, **kwargs)
    global pbar

    pbar = tqdm(position=0, leave=True, smoothing=0.05, desc='Optimization Runs', total=total)
    cerebro.optcallback(optimizer_callbacks)

    result = cerebro.run(optreturn=False, maxcpus=4)
    print('')
    res_list = [[x[0].params.atrPeriodo,
                 x[0].params.atrMult,
                 x[0].params.stopLoss,
                 x[0].params.takeProfit,
                 x[0].analyzers.trade.get_analysis()['total']['total'],
                 abs(x[0].analyzers.trade.get_analysis()['won']['pnl']['total']) / abs(
                     x[0].analyzers.trade.get_analysis()['lost']['pnl']['total']),
                 x[0].analyzers.drawdown.get_analysis()['max']['drawdown'],
                 x[0].analyzers.trade.get_analysis()['long']['won'] / x[0].analyzers.trade.get_analysis()['long'][
                     'total'],
                 x[0].analyzers.trade.get_analysis()['short']['won'] / x[0].analyzers.trade.get_analysis()['short'][
                     'total']
                 ] for x in result]

    pd.set_option("max_columns", None)
    pd.set_option('display.max_rows', None)
    pd.set_option('display.width', 300)

    dfOpt = pd.DataFrame(res_list,
                         columns=['atrPeriodo', 'atrMult', 'stopLoss', 'takeProfit', 'tot_trade', 'ProfitFactor', 'DDMax',
                                  'Win%Long', 'Win%Short'])

    dfOpt = dfOpt.sort_values(['ProfitFactor','tot_trade', 'DDMax', 'Win%Long', 'Win%Short'],
                              ascending=[False, False, True, False, False])
    print(dfOpt)

Lancio l'ottimizzazione:

In [ ]:
ottimizzazioneStrategia(stopLoss = list(np.arange(0.0030, 0.0060, 0.001)), 
                        takeProfit = list(np.arange(0.0030, 0.0060, 0.001)), 
                        atrMult = list(np.arange(1.9, 2.1, 0.1)))

Backtest out-of-sample:

In [ ]:

initCapital = 10000
cerebro = bt.Cerebro()

cerebro.broker.setcommission(leverage=30, commission=0.00001, commtype=CommInfoBase.COMM_FIXED)
cerebro.broker.set_slippage_fixed(0.0001)


data = bt.feeds.GenericCSVData(
    dataname='FXCM_EURUSD_1H_20170101_20211013.csv',
    timeframe=bt.TimeFrame.Minutes,
    compression=60,
    fromdate=datetime(2019, 1, 1),
    todate=datetime(2021, 10, 21),
    dtformat=('%Y%m%d'),
    tmformat=('%H%M'),
    date=0,
    time=1,
    open=2,
    high=3,
    low=4,
    close=5,
    volume=6,
    openinterest=-1
)
# Add the data to Cerebro
cerebro.adddata(data)
# Set our desired cash start
cerebro.broker.setcash(initCapital)

cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="trade")
cerebro.addanalyzer(bt.analyzers.TimeReturn, _name="timereturn")

# Run over everything

cerebro.addstrategy(BreakIn)

result = cerebro.run(tradehistory=True)



Statistiche:

In [ ]:
  finalCapital = initCapital + result[0].analyzers.trade.get_analysis()['pnl']['net']['total']

    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("+++++++++++++++++++++ Backtest ++++++++++++++++++++++")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("")
    print("Ordini Totali..........: {}".format(result[0].analyzers.trade.get_analysis()['total']['total']))
    print("Ordini Streak vinti....: {}".format(result[0].analyzers.trade.get_analysis()['streak']['won']['current']))
    print("Ordini Streak persi....: {}".format(result[0].analyzers.trade.get_analysis()['streak']['lost']['current']))
    print("Profit Factor..........: {}".format(
        abs(result[0].analyzers.trade.get_analysis()['won']['pnl']['total']) / abs(
            result[0].analyzers.trade.get_analysis()['lost']['pnl']['total'])))
    print("Drawdown Max...........: {}".format(result[0].analyzers.drawdown.get_analysis()['max']['drawdown']))
    print("Capitale finale........: {}".format(finalCapital))
    print("Profitto %.............: {}".format(((finalCapital - initCapital) / initCapital) * 100))
    print("PNL lorda..............: {}".format(result[0].analyzers.trade.get_analysis()['pnl']['gross']['total']))
    print("PNL netta..............: {}".format(result[0].analyzers.trade.get_analysis()['pnl']['net']['total']))
    print("---------------- Vincite -------------")
    print("Trade totali...........: {}".format(result[0].analyzers.trade.get_analysis()['won']['total']))
    print("Trade PNL totale.......: {}".format(result[0].analyzers.trade.get_analysis()['won']['pnl']['total']))
    print("Trade PNL medio........: {}".format(result[0].analyzers.trade.get_analysis()['won']['pnl']['average']))
    print("Trade PNL massimo......: {}".format(result[0].analyzers.trade.get_analysis()['won']['pnl']['max']))
    print("---------------- Perdite -------------")
    print("Trade totali...........: {}".format(result[0].analyzers.trade.get_analysis()['lost']['total']))
    print("Trade PNL totale.......: {}".format(result[0].analyzers.trade.get_analysis()['lost']['pnl']['total']))
    print("Trade PNL medio........: {}".format(result[0].analyzers.trade.get_analysis()['lost']['pnl']['average']))
    print("Trade PNL massimo......: {}".format(result[0].analyzers.trade.get_analysis()['lost']['pnl']['max']))
    print("----------------- Long ---------------")
    print("Trade totali...........: {}".format(result[0].analyzers.trade.get_analysis()['long']['total']))
    print("Trade vinti............: {}".format(result[0].analyzers.trade.get_analysis()['long']['won']))
    print("Trade persi............: {}".format(result[0].analyzers.trade.get_analysis()['long']['lost']))
    print("Win/Loss Ratio.........: {}".format(
        result[0].analyzers.trade.get_analysis()['long']['won'] / result[0].analyzers.trade.get_analysis()['long'][
            'total']))
    print("Trade PNL totale.......: {}".format(result[0].analyzers.trade.get_analysis()['long']['pnl']['total']))
    print("Trade PNL Medio........: {}".format(result[0].analyzers.trade.get_analysis()['long']['pnl']['average']))
    print("Trade Vincite Totali...: {}".format(result[0].analyzers.trade.get_analysis()['long']['pnl']['won']['total']))
    print(
        "Trade Vincita Media....: {}".format(result[0].analyzers.trade.get_analysis()['long']['pnl']['won']['average']))
    print("Trade Vincita Massima..: {}".format(result[0].analyzers.trade.get_analysis()['long']['pnl']['won']['max']))
    print(
        "Trade Perdita Totali...: {}".format(result[0].analyzers.trade.get_analysis()['long']['pnl']['lost']['total']))
    print("Trade Perdita Media....: {}".format(
        result[0].analyzers.trade.get_analysis()['long']['pnl']['lost']['average']))
    print("Trade Perdita Massima..: {}".format(result[0].analyzers.trade.get_analysis()['long']['pnl']['lost']['max']))
    print("----------------- Short ---------------")
    print("Trade totali...........: {}".format(result[0].analyzers.trade.get_analysis()['short']['total']))
    print("Trade vinti............: {}".format(result[0].analyzers.trade.get_analysis()['short']['won']))
    print("Trade persi............: {}".format(result[0].analyzers.trade.get_analysis()['short']['lost']))
    print("Win/Loss Ratio.........: {}".format(
        result[0].analyzers.trade.get_analysis()['short']['won'] / result[0].analyzers.trade.get_analysis()['short'][
            'total']))
    print("Trade PNL totale.......: {}".format(result[0].analyzers.trade.get_analysis()['short']['pnl']['total']))
    print("Trade PNL Medio........: {}".format(result[0].analyzers.trade.get_analysis()['short']['pnl']['average']))
    print(
        "Trade Vincite Totali...: {}".format(result[0].analyzers.trade.get_analysis()['short']['pnl']['won']['total']))
    print("Trade Vincita Media....: {}".format(
        result[0].analyzers.trade.get_analysis()['short']['pnl']['won']['average']))
    print("Trade Vincita Massima..: {}".format(result[0].analyzers.trade.get_analysis()['short']['pnl']['won']['max']))
    print(
        "Trade Perdita Totali...: {}".format(result[0].analyzers.trade.get_analysis()['short']['pnl']['lost']['total']))
    print("Trade Perdita Media....: {}".format(
        result[0].analyzers.trade.get_analysis()['short']['pnl']['lost']['average']))
    print("Trade Perdita Massima..: {}".format(result[0].analyzers.trade.get_analysis()['short']['pnl']['lost']['max']))